In [1]:
from huggingface_hub import notebook_login
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
#from google.colab import drive
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from thefuzz import fuzz
from thefuzz import process
import dedupe
import pandas_dedupe

/opt/homebrew/Caskroom/miniconda/base/envs/dsi/lib/python3.11/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
#excel_file_path = '/Users/anthonyhakim/DSI/good-food-purchasing/data/raw/CGFP_bulk.xlsx'
#directory_file_path = '/Users/anthonyhakim/DSI/good-food-purchasing/data/CGFP_bulk_holout_set.csv'
#complete_food_data = pd.read_excel(excel_file_path)

In [272]:
#complete_food_data, holdout_set = train_test_split(complete_food_data, test_size=0.2)

In [273]:
#holdout_set.to_csv(directory_file_path, index=False)

In [308]:
#complete_food_data.to_csv('/Users/anthonyhakim/DSI/good-food-purchasing/CGFP_bulk_training_set.csv')

In [2]:
complete_food_data = pd.read_csv('/Users/anthonyhakim/DSI/good-food-purchasing/data/intermediate/CGFP_bulk_training_set.csv')

/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_26567/2423987643.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  complete_food_data = pd.read_csv('/Users/anthonyhakim/DSI/good-food-purchasing/data/intermediate/CGFP_bulk_training_set.csv')


In [3]:
complete_food_data

,Unnamed: 0,Unique Id,Food Product Category,Origin Detail,Level of Processing,Primary Food Product Category,Product Type,# of Units,Volume Per Unit,Total Volume in lbs,Average Cost Per Unit,Total Cost,Nutrition Certifications,Product Name,Food Product Group,Primary Food Product Group,Unnamed: 15
0,55084,mps-1819-all-purchases-11192019-523,Condiments & Snacks,ABBOTT LABORATORIES,Ultra-Processed,Condiments & Snacks,PEDIASURE CHOCOLATE 8Z ??,20,0.25,5.00,42.630,852.60,NaN,"supplement, chocolate",Condiments & Snacks,Condiments & Snacks,NaN
1,27936,nychhc-18-ll-042220-2298,Vegetables,NaN,Whole/Minimally Processed,Vegetables,ICEBERG ROMAINE MIX CHOPPED 2X5 LB,2,10,20.00,28.750,57.50,NaN,"lettuce, blend",Produce,Produce,NaN
2,67028,ccdj-2019-zp-051921-1294,Cheese,SAPUTO CHEESE USA INC,Moderately Processed,Cheese,CHEESE FETA CRUMBLES,2,NaN,20.00,NaN,57.86,NaN,"cheese, feta, crumbled",Milk & Dairy,Milk & Dairy,NaN
3,127560,sfso-2021-pc-053122-324,Legumes,C & F FOODS,Whole/Minimally Processed,Legumes,Beans Dried Pinto Multi-Cleaned,2,NaN,NaN,1182.015,2364.03,NaN,"bean, pinto, dried","Bread, Grains & Legumes","Bread, Grains & Legumes",NaN
4,18634,trusd-1718-allpurchases-ll-62519-391,Beef,INTEGRATED,Ultra-Processed,Beef,CMDY BEEF PATTY W/APPLESAUCE,1554,NaN,21.88,30.240,46992.96,Red Meat,"beef, patty, commodity",Meat,Meat,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123673,48202,osusd-1819-ll-80968,Grain Products,GENERAL MILLS,Ultra-Processed,Grain Products,CEREAL GRANOLA OATSN HNY BLKPK,275,12.5,3437.50,39.000,10725.00,Whole Grain Rich; Grain Based,"cereal, granola, oats and honey","Bread, Grains & Legumes","Bread, Grains & Legumes",column Production Location: 'GENERAL MILLS' lo...
123674,131438,nychhc-2021-all-purchases-se-090722-1815,"Turkey, Other Poultry",JENNIE-O TURKEY STORE SALES LL / JENNIE-O TURK...,Ultra-Processed,"Turkey, Other Poultry","SAUSAGE, TRKY PTY 1.6 Z 2.9 (4189072)",8,NaN,72.00,NaN,188.40,NaN,"turkey, sausage, patty, raw, frozen",Meat,Meat,NaN
123675,68611,ccdoc-2019-zp-072121-196,Beverages,GRN MTN,Whole/Minimally Processed,Beverages,COFFEE GRND KCUP DARK MAGIC,1,NaN,2.40,NaN,45.11,Healthy Beverage,"coffee, ss",Beverages,Beverages,NaN
123676,12122,mps-1617-allpurchases-dl-62419-565,Grain Products,Pan-o-Gold,Ultra-Processed,Grain Products,4133 - DICKS ST WHEAT -D,17,NaN,NaN,2.030,34.51,Grain Based,"bread, wheat","Bread, Grains & Legumes","Bread, Grains & Legumes",NaN


In [4]:
btype_data = complete_food_data[['Product Type', 'Food Product Group', 'Product Name']]
btype_data.rename(columns={'Product Type': 'text', 'Food Product Group': 'food_category', 'Product Name': 'labels'}, inplace=True)
btype_data['labels'] = btype_data['labels'].str.lower().str.split(',').str[0].str.strip()
btype_data.dropna(inplace=True)
btype_data.reset_index(drop=True, inplace=True)
btype_data['labels'] = btype_data['labels'].str.replace('prequalified: ', '')

/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_26567/4145565903.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btype_data.rename(columns={'Product Type': 'text', 'Food Product Group': 'food_category', 'Product Name': 'labels'}, inplace=True)
/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_26567/4145565903.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btype_data['labels'] = btype_data['labels'].str.lower().str.split(',').str[0].str.strip()
/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_26567/4145565903.py:4: SettingWit

In [81]:
food_category = btype_data['food_category'].unique()
food_category

array(['Condiments & Snacks', 'Produce', 'Milk & Dairy',
       'Bread, Grains & Legumes', 'Meat', 'Beverages', 'Meals',
       'Non-Food', 'Seafood'], dtype=object)

In [82]:
btype_data.labels.unique().__len__()

761

In [5]:
def group_labels(dataframe: pd.DataFrame, category: str, category_label: str, labels_column = 'labels') -> dict:
    
    #subset dataframe by category
    column_name = dataframe[dataframe[category] == category_label][[labels_column]] 
    column_name['value_counts_column'] = column_name['labels'].map(column_name['labels'].value_counts())
    column_name = column_name.sort_values('value_counts_column', ascending=False).reset_index(drop=True)
    
    #shuffle data and transform to set
    # column_name = column_name.sample(frac=1).reset_index(drop=True)
    # column_name = column_name.values.flatten().tolist()
    # column_name = list(set(column_name))
    
    column_name = column_name.labels.values.flatten().tolist()
    print('number of non unique entries in column', len(column_name))
    
    #Create list of dictionaries of subgroups
    label_groups = {}
    for label_entry in column_name:
        counter = 0
        for label_group_key in label_groups.keys():
            if fuzz.ratio(label_entry, label_group_key) > 75:
                counter += 1
                if label_entry != label_group_key:
                    label_groups[label_group_key].append(label_entry)
        if counter == 0:
            label_groups[label_entry] = []
            
    return label_groups

In [6]:
food_dict = {}
for category_label in btype_data['food_category'].unique():
    food_dict[category_label] = group_labels(dataframe= btype_data,category='food_category', category_label=category_label)

number of non unique entries in column 30591
number of non unique entries in column 37205
number of non unique entries in column 8126
number of non unique entries in column 11127
number of non unique entries in column 11298
number of non unique entries in column 7232
number of non unique entries in column 10328
number of non unique entries in column 6240
number of non unique entries in column 1520


In [8]:
food_dict['Produce']

{'pepper': ['peppers'],
 'lettuce': [],
 'apple': [],
 'tomato': ['tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tamato',
  'tamato'],
 'carrot': ['carrots', 'ca

In [85]:
label_sum = 0 
for category_label in btype_data['food_category'].unique():
    print(category_label, 'group size is', len(food_dict[category_label]))
    label_sum += len(food_dict[category_label])
    print('running sum', label_sum)

label_sum

Condiments & Snacks group size is 222
running sum 222
Produce group size is 202
running sum 424
Milk & Dairy group size is 25
running sum 449
Bread, Grains & Legumes group size is 97
running sum 546
Meat group size is 52
running sum 598
Beverages group size is 28
running sum 626
Meals group size is 194
running sum 820
Non-Food group size is 6
running sum 826
Seafood group size is 43
running sum 869


869

In [24]:
print('cod v code: same')
print(fuzz.ratio('code', 'cod'))
print(fuzz.QRatio('code', 'cod'))
print(fuzz.WRatio('code', 'cod'))
print(fuzz.partial_ratio('code', 'cod'))
print(fuzz.token_sort_ratio('code', 'cod'))
print(fuzz.token_set_ratio('code', 'cod'))

print('tilapia v talpia: same')
print(fuzz.ratio('tilapia', 'talpia'))
print(fuzz.QRatio('tilapia', 'talpia'))
print(fuzz.WRatio('tilapia', 'talpia'))
print(fuzz.partial_ratio('tilapia', 'talpia'))
print(fuzz.token_sort_ratio('tilapia', 'talpia'))
print(fuzz.token_set_ratio('tilapia', 'talpia'))

print('apple v pineapple: diff')
print(fuzz.ratio('apple', 'pineapple'))
print(fuzz.QRatio('apple', 'pineapple'))
print(fuzz.WRatio('apple', 'pineapple'))
print(fuzz.partial_ratio('apple', 'pineapple'))
print(fuzz.token_sort_ratio('apple', 'pineapple'))
print(fuzz.token_set_ratio('apple', 'pineapple'))

print('meatball v meat: diff')
print(fuzz.ratio('meatball', 'meat'))
print(fuzz.QRatio('meatball', 'meat'))
print(fuzz.WRatio('meatball', 'meat'))
print(fuzz.partial_ratio('meatball', 'meat'))
print(fuzz.token_sort_ratio('meatball', 'meat'))
print(fuzz.token_set_ratio('meatball', 'meat'))

print('pea v pear: diff')
print(fuzz.ratio('pea', 'pear'))
print(fuzz.QRatio('pea', 'pear'))
print(fuzz.WRatio('pea', 'pear'))
print(fuzz.partial_ratio('pea', 'pear'))
print(fuzz.token_sort_ratio('pea', 'pear'))
print(fuzz.token_set_ratio('pea', 'pear'))

cod v code: same
86
86
86
100
86
86
tilapia v talpia: same
77
77
77
73
77
77
apple v pineapple: diff
71
71
90
100
71
71
meatball v meat: diff
67
67
90
100
67
67
pea v pear: diff
86
86
86
100
86
86


In [86]:
dedupe_df = btype_data[['text','labels']]

There exists intersections in labels. For example almond's top level category can be Condiments & Snacks or Bread, Grains & Legumes.

In [89]:
def find_best_match(misspelled_value, choices):
    best_match = process.extractOne(misspelled_value, choices)
    return best_match[0]

In [90]:
meat = set(food_dict['Meat'].keys())
produce = set(food_dict['Produce'].keys())
condiments_snacks = set(food_dict['Condiments & Snacks'])
milk_dairy = set(food_dict['Milk & Dairy'])
beverages = set(food_dict['Beverages'])
bread_grain_legumes = set(food_dict['Bread, Grains & Legumes'])
meals = set(food_dict['Meals'])
seafood = set(food_dict['Seafood'])
nonfood = set(food_dict['Non-Food'])

In [91]:
def alter_food_set(food_set, add_list, rm_list):
    print('number of elements in food set before additions', len(food_set))
    
    food_set.update(add_list)
    
    food_set.difference_update(rm_list)
    
    print('number of elements in food set after additions', len(food_set))

In [94]:
food_dict['Seafood']

{'tuna': [],
 'pollock': [],
 'shrimp': [],
 'salmon': [],
 'cod': ['code'],
 'tilapia': ['talpia',
  'talpia',
  'talpia',
  'talpia',
  'talpia',
  'talpia',
  'talpia',
  'talpia',
  'talpia',
  'talpia',
  'talpia',
  'talpia',
  'talpia',
  'talpia',
  'talapia',
  'talapia',
  'talapia'],
 'fish': [],
 'catfish': ['crawfish', 'crawfish'],
 'crab cake': [],
 'pangasius': [],
 'crab': [],
 'clam': [],
 'flounder': [],
 'sole': [],
 'snapper': [],
 'trout': [],
 'lobster': [],
 'sardine': [],
 'scallop': [],
 'seafood': [],
 'mahi mahi': [],
 'crabmeat': [],
 'calamari': [],
 'mackerel': [],
 'whiting': [],
 'barramundi': [],
 'mussel': [],
 'anchovy': [],
 'octopus': [],
 'sea bass': [],
 'haddock': [],
 'perch': [],
 'bass': [],
 'halibut': [],
 'caviar': [],
 'walleye': [],
 'char': [],
 'rockfish': [],
 'mix': [],
 'tobiko': [],
 'redfish': [],
 'ramen': [],
 'grenadier': []}

# 1. Meat

In [34]:
meat_add_list = []
meat_rm_list = []
alter_food_set(meat, meat_add_list, meat_rm_list)

number of elements in food set before additions 52
number of elements in food set after additions 52


# 2. Produce

In [35]:
produce_add_list = ['tomatillo',
                    'broccolini',
                    'broccoli rabe',
                    'vegetable cup',
                    'root vegetable',
                    'vegetable mix',
                    'pea',
                    'green pea',
                    'raspberry',
                    'chile',
                    'mustard greens',
                    'palm leaf',
                    'ti leaf']
produce_rm_list = ['mustard green']

alter_food_set(produce, produce_add_list, produce_rm_list)

number of elements in food set before additions 202
number of elements in food set after additions 214


# 3. Condiments Snacks

In [36]:
condiments_snacks_add_list = ['seasoning salt', 
                              'doughnut',
                              'cream', 'tomatillo',
                              'butter','sherbet',
                              'fruit cup','scone',
                              'ice cream cone',]
condiments_snacks_rm_list = []
alter_food_set(condiments_snacks, condiments_snacks_add_list, condiments_snacks_rm_list)

number of elements in food set before additions 222
number of elements in food set after additions 231


# 4. Milk Dairy

In [37]:
milk_dairy_add_list = ['cream']
milk_dairy_rm_list = []
alter_food_set(milk_dairy, milk_dairy_add_list, milk_dairy_rm_list)

number of elements in food set before additions 25
number of elements in food set after additions 26


# 5. Beverages

In [38]:
beverages_add_list = []
beverages_rm_list = []
alter_food_set(beverages, beverages_add_list, beverages_rm_list)

number of elements in food set before additions 28
number of elements in food set after additions 28


# 6. Bread Grain Legumes

In [39]:
bread_grain_legumes_add_list = ['scone']
bread_grain_legumes_rm_list = []
alter_food_set(bread_grain_legumes, bread_grain_legumes_add_list, bread_grain_legumes_rm_list)

number of elements in food set before additions 97
number of elements in food set after additions 98


# 7. Meals

In [40]:
meals_add_list = ['entree', 'pasta salad', 
                  'pretzel dog', 'baked potato', 
                  'egg substitute', 'egg alternative']
meals_rm_list = ['entrée']
alter_food_set(meals, meals_add_list, meals_rm_list)

number of elements in food set before additions 194
number of elements in food set after additions 199


# 8. Seafood

In [41]:
seafood_add_list = ['crawfish']
seafood_rm_list = []
alter_food_set(seafood, seafood_add_list, seafood_rm_list)

number of elements in food set before additions 43
number of elements in food set after additions 44


# 9. Non food

In [75]:
nonfood_add_list = []
nonfood_rm_list = []
alter_food_set(nonfood, nonfood_add_list, nonfood_rm_list)

number of elements in food set before additions 6
number of elements in food set after additions 6


In [43]:
basic_type = meat.union(produce, condiments_snacks, milk_dairy, beverages, bread_grain_legumes, meals, seafood, nonfood)

In [44]:
len(basic_type)

696

In [45]:
meat_df = btype_data[btype_data['food_category'] == 'Meat']
produce_df = btype_data[btype_data['food_category'] == 'Produce']
condiments_snacks_df = btype_data[btype_data['food_category'] == 'Condiments & Snacks']
milk_dairy_df = btype_data[btype_data['food_category'] == 'Milk & Dairy']
beverages_df = btype_data[btype_data['food_category'] == 'Beverages']
bread_grain_legumes_df = btype_data[btype_data['food_category'] == 'Bread, Grains & Legumes']
meals_df = btype_data[btype_data['food_category'] == 'Meals']
seafood_df = btype_data[btype_data['food_category'] == 'Seafood']
nonfood_df = btype_data[btype_data['food_category'] == 'Non-Food']

In [46]:
meals_df

,text,food_category,labels
13,BACON BIT IMITATION,Meals,meatless
14,JAMWICH PB/J STRAW W G,Meals,sandwich
18,"APPETIZER, FRNK IN BLKT .55 Z",Meals,frank in blanket
32,WG CINN FRENCH TOAST STK,Meals,french toast
43,"POTATO, PANCAKE 2 OZ TFF PARFRIED FROZEN",Meals,potato
...,...,...,...
123626,ENTREE CHICKEN TIKKA MASALA,Meals,tikka masala
123627,"POTSTICKER, CHIX VEG 1 Z RAW",Meals,potsticker
123652,IPM Lunch Beef Ribeque Sweet P,Meals,entrée
123654,ANYTIMERS TURKEY HAM & CHS KIT 10206,Meals,meal kit


In [47]:
meat_df['correct_labels'] = meat_df['labels'].apply(lambda x: find_best_match(x, meat))
print('1')
produce_df['correct_labels'] = produce_df['labels'].apply(lambda x: find_best_match(x, produce))
print('2')
condiments_snacks_df['correct_labels'] = condiments_snacks_df['labels'].apply(lambda x: find_best_match(x, condiments_snacks))
print('3')
milk_dairy_df['correct_labels'] = milk_dairy_df['labels'].apply(lambda x: find_best_match(x, milk_dairy))
print('4')
beverages_df['correct_labels'] = beverages_df['labels'].apply(lambda x: find_best_match(x, beverages))
print('5')
bread_grain_legumes_df['correct_labels'] = bread_grain_legumes_df['labels'].apply(lambda x: find_best_match(x, bread_grain_legumes))
print('6')
meals_df['correct_labels'] = meals_df['labels'].apply(lambda x: find_best_match(x, meals))
print('7')
seafood_df['correct_labels'] = seafood_df['labels'].apply(lambda x: find_best_match(x, seafood))
print('8')
nonfood_df['correct_labels'] = nonfood_df['labels'].apply(lambda x: find_best_match(x, nonfood))
print('9')

/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_42380/3619392293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meat_df['correct_labels'] = meat_df['labels'].apply(lambda x: find_best_match(x, meat))


1


/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_42380/3619392293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  produce_df['correct_labels'] = produce_df['labels'].apply(lambda x: find_best_match(x, produce))


2


/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_42380/3619392293.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condiments_snacks_df['correct_labels'] = condiments_snacks_df['labels'].apply(lambda x: find_best_match(x, condiments_snacks))


...

/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_42380/3619392293.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meals_df['correct_labels'] = meals_df['labels'].apply(lambda x: find_best_match(x, meals))


7


/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_42380/3619392293.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seafood_df['correct_labels'] = seafood_df['labels'].apply(lambda x: find_best_match(x, seafood))


8
9


/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_42380/3619392293.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonfood_df['correct_labels'] = nonfood_df['labels'].apply(lambda x: find_best_match(x, nonfood))


In [67]:
print(len(meat_df))
print(len(produce_df))
print(len(condiments_snacks_df))
print(len(milk_dairy_df))
print(len(beverages_df))
print(len(bread_grain_legumes_df))
print(len(meals_df))
print(len(seafood_df))
print(len(nonfood_df))


11298
37205
30591
8126
7232
11127
10328
1520
6240


In [72]:
meat_df

,text,food_category,labels,correct_labels
4,CMDY BEEF PATTY W/APPLESAUCE,Meat,beef,beef
21,Chicken Leg Quarter IQF 60/8.7 Oz 1/32 Lb (791...,Meat,chicken,chicken
34,FRANK ALL-MEAT 8X1 6 IN,Meat,beef,beef
40,CHICKEN COOKED STRIPS BREAST MEAT GRILLED RED ...,Meat,chicken,chicken
41,"EGG, HARD COOKED PEELED WHOLE REF DRY PILLOW P...",Meat,egg,egg
...,...,...,...,...
123606,CHIX NUGGETS WM CNE SO???,Meat,chicken,chicken
123615,TURKEY BOLOGNA SLICED 0.5,Meat,turkey,turkey
123624,"SAUSAGE, TURKEY PATTY 1.5 OZ COOKED NATURAL RA...",Meat,turkey,turkey
123646,"BEEF, STEAK STRIP LOIN CC CHOICE 1180A 1 TAIL...",Meat,beef,beef


In [69]:
basic_type_df = pd.concat([meat_df, produce_df, condiments_snacks_df, milk_dairy_df, beverages_df, bread_grain_legumes_df, meals_df, seafood_df, nonfood_df])
basic_type_df = shuffle(basic_type_df)
btype_dedupe_df = basic_type_df[['text', 'correct_labels']]
btype_dedupe_df.rename(columns={'correct_labels': 'labels'}, inplace=True)

/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_42380/1467162463.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btype_dedupe_df.rename(columns={'correct_labels': 'labels'}, inplace=True)


In [70]:
btype_dedupe_df

,text,labels
110633,CHIP CORN ORIG SS,chip
13209,"SPINACH, CHOPPED FROZEN",spinach
44833,MANDARINS MURCOTT STEM & LEAF OR8B,mandarin
29609,Banana (Petite) GT 188,banana
100588,Sauce Kung Pao,sauce
...,...,...
117486,"PASTA, ROTINI SHELF STABLE SMALL",pasta
45611,CUP FOAM 6OZ SYSCO 40 / 25CT 88811,0
43379,SAUCE OYSTER,sauce
68982,CHEESE MOZZARELLA SHRED VEGAN,cheese alternative


In [52]:
len(btype_dedupe_df.labels.unique())

696

In [307]:
#dedupe_data_file_path = '/Users/anthonyhakim/DSI/good-food-purchasing/basic_type_deduped.csv'
#btype_dedupe_df.to_csv(dedupe_data_file_path, index=False)

In [63]:
basic_type_df

,text,food_category,labels,correct_labels
116623,CHUTNEY MAJOR GRAY,Condiments & Snacks,chutney,chutney
48063,BLEACH LIQ GRMCDL ULTRA 6%,Non-Food,0,0
6191,CARNITA PORK CHPD 4-5# BROOKWOOD,Meat,pork,pork
106805,"MAYONNAISE, EXTRA-HEAVY POUCH SHELF STABLE",Condiments & Snacks,condiment,condiment
32976,SOUP TOMATO BASIL HRVST CONC,Meals,soup,soup
...,...,...,...,...
84538,"THERMOMETER, REFRIGERATOR FREEZER DIAL -20/80F",Non-Food,0,0
35510,CMDY PIZZA PEPPERONI RIPPERS,Meals,pizza,pizza
118605,HARD BOILED EGGS 2 PACK/12CT,Meat,egg,egg
81961,"GRITS, CORN WHT QUICK CNSTR","Bread, Grains & Legumes",grits,grits


In [64]:
inforich_btype_dedupe_df = basic_type_df[['text', 'food_category', 'correct_labels']]
inforich_btype_dedupe_df.rename(columns={'correct_labels': 'labels'}, inplace=True)
inforich_btype_dedupe_df.reset_index(drop=True, inplace=True)

/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_42380/4034916829.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inforich_btype_dedupe_df.rename(columns={'correct_labels': 'labels'}, inplace=True)


In [95]:
inforich_btype_dedupe_df

,text,food_category,labels
0,CHUTNEY MAJOR GRAY,Condiments & Snacks,chutney
1,BLEACH LIQ GRMCDL ULTRA 6%,Non-Food,0
2,CARNITA PORK CHPD 4-5# BROOKWOOD,Meat,pork
3,"MAYONNAISE, EXTRA-HEAVY POUCH SHELF STABLE",Condiments & Snacks,condiment
4,SOUP TOMATO BASIL HRVST CONC,Meals,soup
...,...,...,...
123662,"THERMOMETER, REFRIGERATOR FREEZER DIAL -20/80F",Non-Food,0
123663,CMDY PIZZA PEPPERONI RIPPERS,Meals,pizza
123664,HARD BOILED EGGS 2 PACK/12CT,Meat,egg
123665,"GRITS, CORN WHT QUICK CNSTR","Bread, Grains & Legumes",grits


In [105]:
inforich_btype_dedupe_df[inforich_btype_dedupe_df['food_category'] == 'Non-Food']

,text,food_category,labels
1,BLEACH LIQ GRMCDL ULTRA 6%,Non-Food,0
20,"CONTAINER, PAPER BOARD #3 CATERING BOX SINGLE ...",Non-Food,0
21,TRAY FOOD PAPR RED PLAID 3LB,Non-Food,0
101,LINER PAN QUILLION TRTD 16X24,Non-Food,0
107,"BAG, GROCERY 20 LB PAPER KRAFT BROWN SHORT REC...",Non-Food,0
...,...,...,...
123564,LID PLAS WHITE DOME FOR 8 OZ,Non-Food,0
123572,"MOP HEAD, WET MOP 16 OZ COTTON YARN NARROW BAN...",Non-Food,0
123615,"LABEL, USE BY 2X3 FOOD ROTATION DISSOLVABLE 10...",Non-Food,0
123620,BRILLO PADS HOTEL SIZE,Non-Food,0


In [104]:
inforich_dedupe_data_file_path = '/Users/anthonyhakim/DSI/good-food-purchasing/data/intermediate/inforich_basic_type_deduped.csv'
inforich_btype_dedupe_df.to_csv(inforich_dedupe_data_file_path, index=False)